In [1]:
import pandas as pd
import numpy as np
import bitcoin
import datetime
from ping3 import ping
import random
import json
import re
import multiprocess as mp

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

'/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/data'

In [3]:
def gen_addr():
    # 生成一个随机的密钥
    while True:
        # 生成一个用十六进制表示的长 256 位的私钥（str类型）
        private_key = bitcoin.random_key()
        # 解码为十进制的整形密钥
        decoded_private_key = bitcoin.decode_privkey(private_key, 'hex')
        if 0 < decoded_private_key < bitcoin.N:
            break
    #print(private_key) #密钥（十六进制,长 256 位）

    # 计算地址
    # 用 WIF 格式编码密钥
    wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, 'wif')
    # 用 01 标识的压缩密钥
    compressed_private_key = private_key + '01'
    # 生成 WIF的压缩格式
    wif_compressed_private_key = bitcoin.encode_privkey(
        bitcoin.decode_privkey(compressed_private_key, 'hex'), 'wif')
    # 计算公钥坐标 K = k * G
    public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
    # 计算公钥
    hex_encoded_public_key = bitcoin.encode_pubkey(public_key, 'hex')
    # 计算压缩公钥
    # if public_key[1] % 2 == 0:  # 两种方式均可
    if public_key[1] & 1 == 0:
        compressed_prefix = '02'
    else:
        compressed_prefix = '03'
    # 转十六也可用 bitcoin.encode(xxx, 16)
    hex_compressed_public_key = compressed_prefix + hex(public_key[0])[2:]
    # print(f'压缩公钥（十六进制）{hex_compressed_public_key} '
    #      '（02 开头代表 y 是偶数，03 开头代表 y 是奇数）')
    # 传入公钥坐标对象/十六进制公钥值，输出同样的地址
    address = bitcoin.pubkey_to_address(public_key)
    # print(address) #地址（b58check）（1 开头
    return address

In [6]:
df_user=pd.DataFrame(columns=["user_id", "address"])
for i in range(2000):
    df_user.loc[i]=[i, gen_addr()]
    
df_user

,user_id,address
0,0,1F5AanvjwqYPLwpwHddXLkFb1BgN7jDwbZ
1,1,12b3pjk1tqTEcvxUCUoD1ezsinoB1FYQfc
2,2,13PFdvbCgKaNtw3QZHPvETgU4WGvUmhQ7z
3,3,1K8WbJh6GubEbMACX5cn4vcbowtSzms7Tb
4,4,1QCCQB2a9JSjrjD5uMrhrEJk8ToNr9gb6j
...,...,...
1995,1995,13kS72F4DzeH1fm2a2ztJuDXe8UBG4Npq3
1996,1996,13NX1pVMTtfk6JAszu5Hrd5XzXyUhvZTZ3
1997,1997,1JM7nPTagafsMwGAkTYFJn94nctfkRRPvS
1998,1998,1NfyEbuH2rZ84oTr5myARfNfPC6z7V7BG3


In [7]:
df_user.to_csv("user.csv", index=False)

---

In [26]:
with open("./website.json", "r") as f:
    website_list=json.load(f)

website_list[:10]

[{'id': '2205',
  'name': '人人网',
  'logo': 'http://reg007.u.qiniudn.com/app/renren.jpg',
  'desc': '人人网，中国领先的实名制SNS社交网络。加入人人网，找到老同学，结识新朋友。',
  'category': '社交',
  'home': 'http://www.renren.com/'},
 {'id': '2217',
  'name': '途牛旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/tuniu.jpg',
  'desc': '途牛旅游网-中国专业全面的旅游网,客户满意度94%;提供自助游(自由行),国内旅游,出境旅游,自驾游,公司旅游。低价保证,专业服务,九大出游保障,服务超百万人出游。',
  'category': '旅行',
  'home': 'http://www.tuniu.com/'},
 {'id': '2224',
  'name': '酷讯旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/kuxun.jpg',
  'desc': '酷讯旅游网(Kuxun.cn)是中国领先的在线旅游媒体。为您提供低价可靠的国内国际机票，特价机票99元起，另提供全国20万家酒店预订查询、酒店评论，最低2折起，以及列车时刻表、50000多旅游景点的旅游攻略信息，为您的度假提供最佳的旅游线路。',
  'category': '旅行',
  'home': 'http://www.kuxun.cn/'},
 {'id': '2246',
  'name': '金山词霸',
  'logo': 'http://reg007.u.qiniudn.com/app/iciba.jpg',
  'desc': '爱词霸英语为广大网友提供在线翻译、在线词典、英语口语、英语学习资料、汉语词典，金山词霸下载等服务，致力于为您提供优质权威的在线英语服务，是5000万英语学习者的首选。',
  'category': '工具',
  'home': 'http://www.iciba.com/'},
 {'id': '2247',
  'name': '一听音乐网',
  '

In [27]:
pattern=re.compile("http://(.+?)/")
url_list=[re.findall(pattern, dic["home"])[0] for dic in website_list]
    
url_list[:10]

['www.renren.com',
 'www.tuniu.com',
 'www.kuxun.cn',
 'www.iciba.com',
 'www.1ting.com',
 'www.youku.com',
 'www.zol.com.cn',
 'www.baihe.com',
 'www.izhenxin.com',
 'www.taobao.com']

In [4]:
url=random.choice(url_list)
delay=ping(url)

url
delay

'www.paidai.com'

False

In [5]:
df_user=pd.read_csv("./user.csv")

user_list=df_user["address"].values

In [6]:
# df_tx=pd.DataFrame(columns=["send_timestamp", "recv_timestamp", "latency", "label", "from", "to", "value"])

# latency=None
# last_recv_ts=datetime.datetime(1, 1, 1, 1, 1, 1)
# for _ in range(50000):
#     send_ts=datetime.datetime.now()
    
#     while not latency:
#         url=random.choice(url_list)
#         latency=ping(url)
    
#     recv_ts=send_ts+datetime.timedelta(seconds=latency)
    
#     if recv_ts<last_recv_ts:
#         label=1
#     else:
#         label=0
#     last_recv_ts=recv_ts
    
#     from_user=random.choice(user_list)
#     to_user=random.choice(user_list)
#     value=random.random()*10
    
#     df_tx.loc[len(df_tx)]=[send_ts, recv_ts, latency, label, from_user, to_user, value]
    
# df_tx

In [7]:
def gen_tx():
    send_ts=datetime.datetime.now()
    
    latency=None
    while not latency:
        url=random.choice(url_list)
        latency=ping(url)
    
    recv_ts=send_ts+datetime.timedelta(seconds=latency)
    
    from_user=random.choice(user_list)
    to_user=random.choice(user_list)
    value=random.random()*10
    
    label=0
    return [send_ts, recv_ts, latency, label, from_user, to_user, value]

In [15]:
core=8
pool=mp.get_context("fork").Pool(core)
tx_list=[]
for _ in range(200):
    res=pool.apply_async(gen_tx)
    tx_list.append(res)
    
pool.close()
pool.join()

tx_list=[res.get() for res in tx_list]
len(tx_list)
tx_list[:5]

200

[[datetime.datetime(2022, 10, 11, 16, 27, 28, 809307),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 840556),
  0.031249046325683594,
  0,
  '1A8H4q6CXZmoxPaZ7GfTBY1GhKwuZtUp9o',
  '13kS72F4DzeH1fm2a2ztJuDXe8UBG4Npq3',
  3.0682624454924676],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 814459),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 823095),
  0.008635997772216797,
  0,
  '14Jf3zXzGVMmhMrFJ1zjw75pwtvpAeiWgH',
  '1FYYQsiyFTQgD1qdda4SUEysPPuzqSGoZn',
  3.4326102316053095],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 814585),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 823377),
  0.00879216194152832,
  0,
  '15CD7rWw1ygMGUPxh8JJY6RkEwrhzpZs2e',
  '1AUXQg9G54XVtv1nZRo3Jz726gVkUFQKxW',
  9.103579693156497],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 814710),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 820157),
  0.005446910858154297,
  0,
  '17vuyw1UJrSaowKGS6iPcKv4YDhj3S9rWG',
  '1HkYwEK4s8tAxs162M8VwBhAj7QsWb31fB',
  8.614325514475862],
 [datetime.datetime(2022, 10, 11, 1

In [16]:
tx_list_sorted=sorted(tx_list, key=lambda tx: tx[1])

tx_list_sorted

[[datetime.datetime(2022, 10, 11, 16, 27, 28, 814710),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 820157),
  0.005446910858154297,
  0,
  '17vuyw1UJrSaowKGS6iPcKv4YDhj3S9rWG',
  '1HkYwEK4s8tAxs162M8VwBhAj7QsWb31fB',
  8.614325514475862],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 815063),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 822302),
  0.007238864898681641,
  0,
  '1F5N4vdyksZcKTVPW9pT1yxBm7wr7XiTiZ',
  '19H3uwDMXaoagXgVphKLAcC81YeJwG2vC8',
  1.3238423776818131],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 814892),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 822632),
  0.0077397823333740234,
  0,
  '1No2aNkrwWPGmB1jAdS3LhFT8VSgNQdgWb',
  '1KVFmMaUTzZuR6PB87mR1jVKFfq2CzFy9D',
  5.284916684757856],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 814459),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 823095),
  0.008635997772216797,
  0,
  '14Jf3zXzGVMmhMrFJ1zjw75pwtvpAeiWgH',
  '1FYYQsiyFTQgD1qdda4SUEysPPuzqSGoZn',
  3.4326102316053095],
 [datetime.datetime(2022, 10, 11,

In [17]:
for i in range(1, len(tx_list_sorted)):
    if tx_list_sorted[i][0]<tx_list_sorted[i-1][0]:
        tx_list_sorted[i][3]=1
        
tx_list_sorted

[[datetime.datetime(2022, 10, 11, 16, 27, 28, 814710),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 820157),
  0.005446910858154297,
  0,
  '17vuyw1UJrSaowKGS6iPcKv4YDhj3S9rWG',
  '1HkYwEK4s8tAxs162M8VwBhAj7QsWb31fB',
  8.614325514475862],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 815063),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 822302),
  0.007238864898681641,
  0,
  '1F5N4vdyksZcKTVPW9pT1yxBm7wr7XiTiZ',
  '19H3uwDMXaoagXgVphKLAcC81YeJwG2vC8',
  1.3238423776818131],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 814892),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 822632),
  0.0077397823333740234,
  1,
  '1No2aNkrwWPGmB1jAdS3LhFT8VSgNQdgWb',
  '1KVFmMaUTzZuR6PB87mR1jVKFfq2CzFy9D',
  5.284916684757856],
 [datetime.datetime(2022, 10, 11, 16, 27, 28, 814459),
  datetime.datetime(2022, 10, 11, 16, 27, 28, 823095),
  0.008635997772216797,
  1,
  '14Jf3zXzGVMmhMrFJ1zjw75pwtvpAeiWgH',
  '1FYYQsiyFTQgD1qdda4SUEysPPuzqSGoZn',
  3.4326102316053095],
 [datetime.datetime(2022, 10, 11,

In [18]:
df_tx=pd.DataFrame(columns=["send_timestamp", "recv_timestamp", "latency", "label", "from", "to", "value"])

for tx in tx_list_sorted:
    df_tx.loc[len(df_tx)]=tx
    
df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-10-11 16:27:28.814710,2022-10-11 16:27:28.820157,0.005447,0,17vuyw1UJrSaowKGS6iPcKv4YDhj3S9rWG,1HkYwEK4s8tAxs162M8VwBhAj7QsWb31fB,8.614326
1,2022-10-11 16:27:28.815063,2022-10-11 16:27:28.822302,0.007239,0,1F5N4vdyksZcKTVPW9pT1yxBm7wr7XiTiZ,19H3uwDMXaoagXgVphKLAcC81YeJwG2vC8,1.323842
2,2022-10-11 16:27:28.814892,2022-10-11 16:27:28.822632,0.007740,1,1No2aNkrwWPGmB1jAdS3LhFT8VSgNQdgWb,1KVFmMaUTzZuR6PB87mR1jVKFfq2CzFy9D,5.284917
3,2022-10-11 16:27:28.814459,2022-10-11 16:27:28.823095,0.008636,1,14Jf3zXzGVMmhMrFJ1zjw75pwtvpAeiWgH,1FYYQsiyFTQgD1qdda4SUEysPPuzqSGoZn,3.432610
4,2022-10-11 16:27:28.814585,2022-10-11 16:27:28.823377,0.008792,0,15CD7rWw1ygMGUPxh8JJY6RkEwrhzpZs2e,1AUXQg9G54XVtv1nZRo3Jz726gVkUFQKxW,9.103580
...,...,...,...,...,...,...,...
195,2022-10-11 16:27:41.516080,2022-10-11 16:27:41.560769,0.044689,0,1F4C17aTtxydvkFTqBZcxp1ioWC6Y6JzC9,1Bw1SzuSTzJrBkGKoQnr2GWjf9CkrqEMGk,3.426720
196,2022-10-11 16:27:41.531272,2022-10-11 16:27:41.572383,0.041111,0,1NmK67kXsYtDAg4WkpDqTCFg4ZfarJCChN,1PoQ8b7mKk6ytXZAKHLrQMUL2ngAep5GLX,3.771443
197,2022-10-11 16:27:41.575261,2022-10-11 16:27:41.627808,0.052547,0,19Z5GCRG8RRdX6etCsQg5YWikrYPyEDpFu,16oYWTTQKmes6iUvbJB4yrexgFGBkQjYvs,2.996111
198,2022-10-11 16:27:41.631741,2022-10-11 16:27:41.689441,0.057700,0,1HxJQtM1t1RY38K8ysoCmZzgDLjJWwcHQn,1J8NYLw7T5w9bbzD7JmC8XahVS5PJZU3p7,2.927839


In [22]:
sum(df_tx["label"]==1)

42

In [25]:
df_tx.to_csv(f"./tx_{len(df_tx)}.csv", index=False)